# Metrics Retrieval - from Clickhouse

On the previous steps we've already:
1. Crawled archives via Airflow
2. Processed them via Airflow -> Spark job(s)
    * When procesing, we've stored daily & hourly datasets into Clickhouse, to be later used for aggregations.

This step in exactly about aggregating data from Clcikhouse to get desired metrics.

Eventually, this fella will be ALSO wrapped into an Airflow job, which will be triggered after the daily Spark job is done.

# Implementation

In [1]:
import clickhouse_connect

# I konw, it's unsafe, and should be rewritten wo use ENVs at least
client = clickhouse_connect.get_client(
    host='clickhouse_server', 
    username='altenar', 
    password='altenar_ch_demo_517'
)

## Use Database for GHArchive Project

In [6]:
client.command('USE gharchive;')

''

In [8]:
print(client.command("show tables from gharchive;"))

commits
members
repo_aggregated
repos
test_table


## Metric 1. Repo Ownership

List of Developers that own more than one repository;

In [14]:
cmd = """
SELECT
    repo_author,
    countDistinct(repo_name) AS repos_total
FROM gharchive.repos
GROUP BY repo_author
HAVING repos_total > 1
ORDER BY repos_total DESC
"""

In [11]:
help(client.query_df)

Help on method query_df in module clickhouse_connect.driver.client:

query_df(query: str = None, parameters: Union[Sequence, Dict[str, Any], NoneType] = None, settings: Optional[Dict[str, Any]] = None, query_formats: Optional[Dict[str, str]] = None, column_formats: Optional[Dict[str, str]] = None, encoding: Optional[str] = None, use_none: Optional[bool] = None, max_str_len: Optional[int] = None, use_na_values: Optional[bool] = None, query_tz: Optional[str] = None, column_tzs: Optional[Dict[str, Union[str, datetime.tzinfo]]] = None, context: clickhouse_connect.driver.query.QueryContext = None, external_data: Optional[clickhouse_connect.driver.external.ExternalData] = None, use_extended_dtypes: Optional[bool] = None) method of clickhouse_connect.driver.httpclient.HttpClient instance
    Query method that results the results as a pandas dataframe.  For parameter values, see the
    create_query_context method
    :return: Pandas dataframe representing the result set



In [12]:
import pandas as pd

In [15]:
df = client.query_df(query=cmd)

In [16]:
df

,repo_author,repos_total
0,direwolf-github,8894
1,pmacik-testing,6659
2,MetadataGitTesting,5145
3,redhat-appstudio-qe,2481
4,acciojob,2011
...,...,...
272663,Ilsha,2
272664,uweremer,2
272665,pwndbg,2
272666,coinos,2


In [17]:
from datetime import datetime, timedelta

In [18]:
now = datetime.now()

In [19]:
df.to_csv("./repos_{}.csv".format(now.strftime("%F")))

## Metric 2. Devs with more than 1 commit

List of Developers who did more than one commit in a day, ordered by name and number of commits;

In [22]:
cmd = """
SELECT                                                                                                                                                                                                                                                                   
    date,
    author_name,
    count(*) AS total_commits
FROM gharchive.commits
GROUP BY
    date,
    author_name
HAVING total_commits >= 2
ORDER BY
    author_name ASC,
    total_commits DESC
"""

In [12]:
import pandas as pd

In [23]:
df = client.query_df(query=cmd)

In [24]:
df

,date,author_name,total_commits
0,2023-07-20,,1976
1,2023-07-21,,1972
2,2023-07-19,,1730
3,2023-07-20,! Lucam,10
4,2023-07-19,! Ruwin.zkr,10
...,...,...,...
850934,2023-07-20,🤖 automated,16
850935,2023-07-19,🤖 automated,13
850936,2023-07-19,🦄Pegasus (DarkStar☀),22
850937,2023-07-20,🦄Pegasus (DarkStar☀),17


In [25]:
now = datetime.now()

In [26]:
df.to_csv("./commits_gt1_{}.csv".format(now.strftime("%F")))

## Metric 3. Devs with __less__ than 1 commit

List of Developers with less than one commit in a day;

In [27]:
# Take min and max datetime from commits - those are ranges by which we'll move

In [41]:
cmd = """
SELECT
    toDate(min(created_at)) AS min_created_at,
    toDate(max(created_at)) AS max_created_at
FROM gharchive.commits
"""

In [42]:
dates_df = client.query_df(query=cmd)

In [43]:
dates_df

,min_created_at,max_created_at
0,2023-07-19,2023-07-21


In [47]:
dates_df['min_created_at'][0].strftime("%F")

'2023-07-19'

In [46]:
dates_df['max_created_at'][0]

Timestamp('2023-07-21 00:00:00')

In [53]:
cmd = """
SELECT t.author_name, t.interval_start_dt
FROM
(
    SELECT
        author_name,
        created_at,
        1 AS total,
        toStartOfInterval(created_at, toIntervalDay(1)) AS interval_start_dt,
        sum(total) OVER (PARTITION BY author_name, interval_start_dt ORDER BY created_at ASC) AS sum_commits
    FROM gharchive.commits
    ORDER BY
        author_name ASC,
        interval_start_dt ASC 
            WITH FILL 
            FROM toUnixTimestamp('{}') 
            TO toUnixTimestamp('{}') 
            STEP toIntervalDay(1)
) AS t
WHERE sum_commits = 0
""".format(
    dates_df['min_created_at'][0].strftime("%F"),
    (dates_df['max_created_at'][0] + timedelta(days=1)).strftime("%F")
)

In [54]:
df = client.query_df(query=cmd)

In [55]:
df

,author_name,interval_start_dt
0,! Lucam,2023-07-19
1,! Lucam,2023-07-21
2,! Ruwin.zkr,2023-07-20
3,! Ruwin.zkr,2023-07-21
4,! Rxin,2023-07-19
...,...,...
1233502,🤖github-actions🍦,2023-07-20
1233503,🤖github-actions🍦,2023-07-21
1233504,🦄Pegasus (DarkStar☀),2023-07-21
1233505,🦉🤖 Safe Bot,2023-07-19


In [56]:
now = datetime.now()

In [57]:
df.to_csv("./commits_lt1_{}.csv".format(now.strftime("%F")))

## Metric 4. Total Developers grouped by gender

Metric is skipped as there is no such data to be referenced from gharchive's datasets.

The only possible way is to crawl each individual's author/github user's pronouns, but that's a chore that definetely requires a serious discussion about epic's requirements.

## Metric 5. Projects with more than 10 members

Total projects with more than 10 members;

__NOTE__!
* We do not consider forks / same repo name across different users as THE SAME project. That results in grouping by repo_name_full, not chunked repo_name.

In [58]:
cmd = """
SELECT
    repo_name,
    repo_name_full,
    countDistinct(member_login) AS total_members
FROM gharchive.members
GROUP BY
    repo_name,
    repo_name_full
HAVING total_members > 10
ORDER BY 
    total_members DESC,
    repo_name ASC
"""

In [59]:
df = client.query_df(query=cmd)

In [60]:
df

,repo_name,repo_name_full,total_members
0,Metodos_numerico,osvinfor/Metodos_numerico,30
1,Parle-Agro,rongalimanikanta/Parle-Agro,30
2,open-source-in-nigeria,adebowale-akinola/open-source-in-nigeria,29
3,AMUL,Poojitha-2822/AMUL,28
4,curso-serasa,MarciusMcflay/curso-serasa,28
5,Prodapt,Karthi-018/Prodapt,27
6,HIMALAYAHERBAL,KDEMUDUHARIPRIYA/HIMALAYAHERBAL,23
7,Himalaya-Herbal,KDEMUDUHARIPRIYA/Himalaya-Herbal,21
8,HIMALAYA-HERBALS,Parameswari2002/HIMALAYA-HERBALS,20
9,Lakme,Balaji-1805/Lakme,20


In [61]:
now = datetime.now()

In [62]:
df.to_csv("./projects_gt10_{}.csv".format(now.strftime("%F")))